In [31]:
import doekbase.data_api
from doekbase.data_api.annotation.genome_annotation.api import GenomeAnnotationAPI
import time
import os

In [91]:
def get_gff(gene,genome_annotation, taxon_id, source_info, count, contig_len):
    
    print(gene)
    #retrieve pairwise inter-feature relationships between genes, mRNAs, CDSs
    #the below section of code has method timings torecord performance  
    t0 = time.time()
    listmrna=genome_annotation.get_mrna_by_gene([gene])[gene]    
    t1 = time.time()
    print "get_mrna_by_gene " , (t1-t0), "s"
    #print(listmrna)
    
    t0 = time.time()
    listcds=genome_annotation.get_cds_by_gene([gene])[gene] 
    t1 = time.time()
    print "get_cds_by_gene " , (t1-t0), "s"   
    print(listcds)
        
    if len(listmrna) > 0:
        t0 = time.time()
        print(listmrna)
        mrna_cds=genome_annotation.get_cds_by_mrna(listmrna)
        t1 = time.time()
        print "get_cds_by_mrna " , (t1-t0), "s"
        #print(mrna_cds)
    
        #t0 = time.time()
        #cds_mrna=genome_annotation.get_mrna_by_cds(listcds)
        #t1 = time.time()
        #print "get_mrna_by_cds " , (t1-t0), "s"

        t0 = time.time()
        get_mrna_exons = genome_annotation.get_mrna_exons(listmrna)
        t1 = time.time()
        print "get_mrna_exons " , (t1-t0), "s"
        #print get_mrna_exons
        
        #retrieve locations for mRNAs
        t0 = time.time()
        mrna_location=genome_annotation.get_feature_locations(listmrna) 
        t1 = time.time()
        print "get_feature_locations " , (t1-t0), "s"


    #retrieve locations for genes, mRNAs, and cds
    t0 = time.time()
    gene_location=genome_annotation.get_feature_locations([gene]) 
    t1 = time.time()
    print "get_feature_locations " , (t1-t0), "s"
    
    if len(listcds) > 0:
        t0 = time.time()
        cds_location=genome_annotation.get_feature_locations(listcds) 
        t1 = time.time()
        print "get_feature_locations " , (t1-t0), "s"
        print cds_location
    
    string=""
    
    if count == 0:
        string+="##sequence-region "+gene_location[gene_location.keys()[0]][0]['contig_id']+"\t"+"1"+"\t"+str(contig_len)+"\n"##NC_003070.9 1 30427671
        if taxon_id != -1:
            string+="##species http://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id="+str(taxon_id)+"\n"#http://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=3702
        else :
            string+="##species unknown\n"
    

    contig_id = ""
    #output the gene data
    for x in gene_location:
        #print(type(x))
        #print(x)
        info=gene_location[x][0]
        #print(type(info))
        #print info
        stop=info['start'] + info['length']
        
        contig_id = info['contig_id']
        
        #print "source_info"
        #print(type(source_info))
        #print(source_info)
        #output aliases for gene as dbxref?
        aliases = genome_annotation.get_feature_aliases([x])
        db_xref = ""
        for a in aliases:
            db_xref = db_xref + " db_xref="+a +";"
        string+=info['contig_id']  + "\t" + source_info['external_source']+"\tgene\t"  + str(info['start']) + "\t" + str(stop) + "\t.\t" +  info['strand']+ "\t.\t" + "ID=" + x + ";" + "Name="+ x + ";" +db_xref+"\n"
    
    print "len(listmrna)"    
    print(len(listmrna))
    if len(listmrna) > 0:    
    #output the mRNA data
        for mrna in listmrna:
            print("mrna "+mrna)
            aliases = genome_annotation.get_feature_aliases([mrna])
            db_xref = ""
            if len(aliases) > 0:
                for a in aliases:
                    db_xref = db_xref + " db_xref="+a +";"
                
            cds=mrna_cds[mrna]
            infomrna=mrna_location[mrna][0]
            #print(type(infomrna))
            #print(infomrna)
            #infocds=cds_location[mrna_cds[mrna]]
            stop=infomrna['start'] + infomrna['length']            
            ###output mrna data
            string+=infomrna['contig_id']  + "\t" +source_info['external_source']+"\tmRNA\t"  + str(infomrna['start'])+ "\t" + str(stop) + "\t.\t" +  infomrna['strand']+ "\t.\t" + "ID=" + mrna + ";" + "Parent="+ gene+";db_xref="+db_xref+"\n"
            
            #print 'get_mrna_exons.keys()'
            #print get_mrna_exons.keys()
            
            ##TODO output UTRs for exons
            ###output exon data
            for i in range(0, len(get_mrna_exons[mrna])):
                #print "mrna, gene"
                #print type(mrna)
                #print type(gene)
                print get_mrna_exons[mrna][i]
                stop= get_mrna_exons[mrna][i]['exon_location']['start'] + get_mrna_exons[mrna][i]['exon_location']['length']

                string+= get_mrna_exons[mrna][i]['exon_location']['contig_id']  + "\t" +source_info['external_source'] + "\texon\t"  + str(get_mrna_exons[mrna][i]['exon_location']['start'])+ "\t" + str(stop) + "\t.\t" +  get_mrna_exons[mrna][i]['exon_location']['strand']+ "\t.\t" + "ID="+mrna+".exon." + str(get_mrna_exons[mrna][i]['exon_ordinal']) + ";" + "Parent="+ mrna+"\n"
            ###output CDS data
            for k in cds_location.keys(): #range(0, len(cds_locations[gene])):
                aliases = genome_annotation.get_feature_aliases([mrna])
                db_xref = ""
                if len(aliases) > 0:
                    for a in aliases:
                        db_xref = db_xref + " db_xref="+a +";"
                for i in range(0,len(cds_location[k])):
                    stop= cds_location[k][i]['start'] + cds_location[k][i]['length']
                    ##***TODO: set PHASE for CDS
                    string+= cds_location[k][i]['contig_id']  + "\t" +source_info['external_source'] +  "\tCDS\t"  + str(cds_location[k][i]['start'])+ "\t" + str(stop) + "\t.\t" +  cds_location[k][i]['strand']+ "\t.\t" + "ID=" + k  + ";" + "Parent="+ mrna+"\n"

    return string       

#hardcoded test with 5 random genes
def runDemo(ws_url='https://ci.kbase.us/services/ws/', ws="ReferenceEnsemblPlantGenomeAnnotations", genome_id="280699_Ensembl"):
    print "1"
    #an example KBase reference genome
    genome_ref = "ReferenceEnsemblPlantGenomeAnnotations"+"/"+genome_id
    #instantiate a new genome annotation API
    t0 = time.time()
    genome_annotation = GenomeAnnotationAPI(services = {"workspace_service_url": ws_url}, token=os.environ.get('KB_AUTH_TOKEN'), ref=genome_ref)
    t1 = time.time()
    print "GenomeAnnotationAPI " , (t1-t0), "s"
    
    t0 = time.time()
    taxon = genome_annotation.get_taxon()
    t1 = time.time()
    print "get_taxon " , (t1-t0), "s"
        
    t0 = time.time()
    taxonid = taxon.get_taxonomic_id()
    t1 = time.time()
    print "get_taxonomic_id " , (t1-t0), "s"
    
    if taxonid == -1 :
        taxonid = "unknown"
    
    assembly = genome_annotation.get_assembly()
    contig_ids = assembly.get_contig_ids()
    sourceinfo = assembly.get_external_source_info()
    #genes=['gene_1','gene_2','gene_5', 'gene_100', 'gene_1000']
    genes=['gene_23420']
    
    gffdata=""
    gffdata+="##gff-version 3\n"
    gffdata+="#!gff-spec-version 1.21\n"
    gffdata+="#!processor KBase GFF3 downloader\n"
    #!genome-build TAIR10
    #!genome-build-accession NCBI_Assembly:GCF_000001735.3   
    
    count = 0
    for s in genes:
        gffdata+=get_gff(s,genome_annotation,taxonid, sourceinfo, count, -1)
        count = count + 1

    gffdata+="###"
    outfile = genome_id+'_demo.gff'
    print outfile
    with open(outfile, 'w') as f:
        f.write(gffdata)
    
#create GFF for an entire GenomeAnnotation
def runGenome(ws_url='https://ci.kbase.us/services/ws/', ws="ReferenceEnsemblPlantGenomeAnnotations", genome_id="280699_Ensembl"):
    #an example KBase reference genome
    genome_ref = "ReferenceEnsemblPlantGenomeAnnotations"+"/"+genome_id#kb|g.166819"
    #instantiate a new genome annotation API
    t0 = time.time()
    genome_annotation = GenomeAnnotationAPI(services = {"workspace_service_url": ws_url}, token=os.environ.get('KB_AUTH_TOKEN'), ref=genome_ref)
    t1 = time.time()
    print "GenomeAnnotationAPI " , (t1-t0), "s"
    
    t0 = time.time()
    taxon = genome_annotation.get_taxon()
    t1 = time.time()  
    print "get_taxon " , (t1-t0), "s"
        
    t0 = time.time()
    taxonid = taxon.get_taxonomic_id()
    t1 = time.time()
    print "get_taxonomic_id " , (t1-t0), "s"
    
    print "1"
    
    if taxonid == -1 :
        taxonid = "unknown"
    
    assembly = genome_annotation.get_assembly()
    contig_ids = assembly.get_contig_ids()
    sourceinfo = assembly.get_external_source_info()
    
    contig_lens = assembly.get_contig_lengths()
    #print contig_lens
    
    print "2"
    
    gffdata=""
    gffdata+="##gff-version 3\n"
    gffdata+="#!gff-spec-version 1.21\n"
    gffdata+="#!processor KBase GFF3 downloader\n"
    #!genome-build TAIR10
    #!genome-build-accession NCBI_Assembly:GCF_000001735.3  
    
    start=1
    stop=100000000
    
    feat_data = []
    print "contig_ids len "+ str(len(contig_ids))
    countc = 0
    feat_total = 0
    for c in contig_ids:
        
        stop = contig_lens[c]
        print c
        regions = []
        contig_add = {"contig_id": c, "start": start, "length": stop-start, "strand": "+"}
        contig_add2 = {"contig_id": c, "start": stop, "length": stop-start, "strand": "-"}
        regions.append(contig_add)
        regions.append(contig_add2)
        
        t0 = time.time()
        feat_data = genome_annotation.get_feature_ids(filters={"region_list":regions,"type_list":["gene"]})['by_type']['gene']
        t1 = time.time()
        print "get_feature_ids " , (t1-t0), "s"
    
        print "feat_data len "+ str(len(feat_data))
        #print feat_data
        #for s in genes:
        countf =0
        feat_total = feat_total + len(feat_data)
        for f in feat_data:
            print f
            gffdata+=get_gff(f,genome_annotation,taxonid, sourceinfo, countf, stop)
            #print gffdata
            countf = countf + 1
            #break after small number       
            if countf > 3:
                print "break after first 3 genes for now"
                break
               
        countc = countc + 1
        if countc > 5:
            print "break after first 3 contigs for now "   
            #break after first contig for now    
            break
    
    #print "END"
    #print gffdata
    
    print "process contigs " + str(countc)
    print "feat total " + str(feat_total)
    
    gffdata+="###"
    outfile = genome_id+'.gff'
    print outfile
    with open(outfile, 'w') as f:
        f.write(gffdata)
        
    feat_data

In [92]:
genomeid="3702_Ensembl"#280699_Ensembl"
wsthis = "ReferenceEnsemblPlantGenomeAnnotations"
###Note that this demo function does not produce biologically valid GFF
runDemo(ws=wsthis, genome_id=genomeid)

1
GenomeAnnotationAPI  1.45173692703 s
get_taxon  3.61539292336 s
get_taxonomic_id  4.05311584473e-06 s
gene_23420
get_mrna_by_gene  4.01392197609 s
get_cds_by_gene  4.26782894135 s
[u'CDS_23955', u'CDS_23954']
[u'mRNA_23954', u'mRNA_23955']
get_cds_by_mrna  3.9410431385 s
get_mrna_exons  4.07889103889 s
get_feature_locations  14.9473378658 s
get_feature_locations  15.5665130615 s
get_feature_locations  15.1564590931 s
{u'CDS_23955': [{'start': 21249288, 'length': 79, 'contig_id': u'chromosome:TAIR10:5:1:26975502:1', 'strand': u'-'}, {'start': 21249118, 'length': 69, 'contig_id': u'chromosome:TAIR10:5:1:26975502:1', 'strand': u'-'}, {'start': 21248979, 'length': 133, 'contig_id': u'chromosome:TAIR10:5:1:26975502:1', 'strand': u'-'}, {'start': 21248702, 'length': 156, 'contig_id': u'chromosome:TAIR10:5:1:26975502:1', 'strand': u'-'}, {'start': 21248394, 'length': 370, 'contig_id': u'chromosome:TAIR10:5:1:26975502:1', 'strand': u'-'}, {'start': 21247917, 'length': 104, 'contig_id': u'chr

In [67]:
genomeid="3702_Ensembl"#280699_Ensembl"
wsthis = "ReferenceEnsemblPlantGenomeAnnotations"
runGenome(ws=wsthis, genome_id=genomeid)

GenomeAnnotationAPI  1.87069296837 s
get_taxon  4.64188194275 s
get_taxonomic_id  9.77516174316e-06 s
1
2
contig_ids len 7
chromosome:TAIR10:3:1:23459830:1
get_feature_ids  30.051074028 s
feat_data len 6730
gene_15036
gene_15036
get_mrna_by_gene  4.0081410408 s
get_cds_by_gene  5.05426096916 s
[]
get_feature_locations  13.1051459312 s
len(listmrna)
0
gene_13981
gene_13981
get_mrna_by_gene  4.9694108963 s
get_cds_by_gene  4.04269599915 s
[u'CDS_14745']
[u'mRNA_14745']
get_cds_by_mrna  3.84603500366 s
get_mrna_exons  3.73147892952 s
get_feature_locations  14.3686590195 s
get_feature_locations  18.195141077 s
get_feature_locations  15.8439760208 s
{u'CDS_14745': [{'start': 9843639, 'length': 20, 'contig_id': u'chromosome:TAIR10:3:1:23459830:1', 'strand': u'+'}, {'start': 9843758, 'length': 81, 'contig_id': u'chromosome:TAIR10:3:1:23459830:1', 'strand': u'+'}, {'start': 9843943, 'length': 269, 'contig_id': u'chromosome:TAIR10:3:1:23459830:1', 'strand': u'+'}, {'start': 9844367, 'length': 5

KeyboardInterrupt: 

In [ ]:
#write overall header
#for contigs
    #get contig features
    #write contig header
    #for gene in contig
        #write GFF gene
            #for mRNA from gene
                #write GFF mRNA for gene
                    #for exon in mRNA
                        #write GFF exon for mRNA
                        #write GFF CDS for mRNA
